In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch as torch
import torch.nn as nn

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from google.colab import files
uploaded = files.upload()

Saving df1_smallestdata.csv to df1_smallestdata.csv
Saving df2_smallestdata.csv to df2_smallestdata.csv
Saving targets_smallestdata.csv to targets_smallestdata.csv


In [37]:
input_x1 = pd.read_csv('df1_smallestdata.csv')
input_x2 = pd.read_csv('df2_smallestdata.csv')
target_y = pd.read_csv('targets_smallestdata.csv')

# Drop the first column
input_x1 = input_x1.drop(input_x1.columns[0], axis=1)
input_x2 = input_x2.drop(input_x2.columns[0], axis=1)

# Convert to numpy
input_x1 = input_x1.to_numpy()
input_x2 = input_x2.to_numpy()
target_y = target_y.to_numpy()

print(input_x1.shape)
print(input_x2.shape)
print(target_y.shape)



(250, 24)
(250, 24)
(250, 1)


In [42]:
# make a neural network with 1 hidden layer using pytorch
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(24, 16)
        self.fc2 = nn.Linear(16, 9)
        self.fc3 = nn.Linear(9, 3)
        self.fc4 = nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.SGD(self.parameters(), lr=0.01)



    def forward(self, x1, x2):
        # Element-wise subtraction
        x = x1 - x2
        # Forward pass with shared weights
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        x = self.fc4(x)
        return x

    def loss(self, output, target):
        return self.criterion(output, target)

    def backward(self, loss):
        loss.backward()

    def train(self, x1, x2, target):
        # Forward pass
        output = self.forward(x1, x2)
        # Compute the loss
        loss = self.loss(output, target)
        # Zero the gradients
        self.optimizer.zero_grad()
        # Backward pass
        self.backward(loss)
        # Update the weights
        self.optimizer.step()
        return loss



In [47]:
model = Net()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
for epoch in range(1000):  # Adjust epochs as needed
  for j in range(len(input_x1)):
    # Move data to GPU (if available)
    x1 = torch.tensor(input_x1[j], dtype=torch.float32)
    x2 = torch.tensor(input_x2[j], dtype=torch.float32)
    target = torch.tensor(target_y[j], dtype=torch.float32)


    x1 = x1.to(device)
    x2 = x2.to(device)
    targets = target.to(device)

    model.train(x1, x2, targets)
